In [4]:
# Importing the required libraries
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.metrics import mean_squared_error


SyntaxError: invalid syntax (pywrap_tensorflow_internal.py, line 114)

In [ ]:
# Defining the list of tickers
tickers = ['AAPL', 'GOOG', 'AMZN', 'TSLA', 'MSFT',"^GSPC", "^IXIC"]

# Collecting the data
data = yf.download(tickers, start='2012-01-01', end='2022-01-01')

In [ ]:

# Cleaning the data
data.dropna(inplace=True)

# Normalizing the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

# Splitting the data into training and testing sets
train_size = int(len(scaled_data) * 0.8)
test_size = len(scaled_data) - train_size
train_data, test_data = scaled_data[0:train_size,:], scaled_data[train_size:len(scaled_data),:]


In [ ]:

# Defining the function to create the dataset with look-back
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:

# Creating the dataset with look-back
look_back = 60
X_train, y_train = create_dataset(train_data, look_back)
X_test, y_test = create_dataset(test_data, look_back)

# Reshaping the input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Defining the LSTM model with layered neural network
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

In [ ]:

# Compiling the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Fitting the model to the training data
model.fit(X_train, y_train, epochs=50, batch_size=32)

In [ ]:
import tensorflow as tf

# Predicting the test data
y_pred = model.predict(X_test)

# Calculating the root mean squared error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', rmse)

# Calculating accuracy
accuracy = tf.keras.metrics.Accuracy()
accuracy.update_state(tf.math.round(y_test), tf.math.round(y_pred))
acc = accuracy.result().numpy()
print('Accuracy:', acc)

In [ ]:

import pickle
  
# Save the trained model as a pickle string.
saved_model = pickle.dumps(model)


from joblib import Parallel, delayed
import joblib
  
  
# Save the model as a pickle in a file
joblib.dump(saved_model, 'model1_lstm_full.pkl')

#model_yaml = model.to_yaml()
#with open("model.yaml", "w") as yaml_file:
#    yaml_file.write(model_yaml)
# serialize weights to HDF5
#model.save_weights("model.h5")
print("Saved model to disk")

# New Section

In [ ]:




# Ensemble approach with Random Forest, Gradient Boosting, and AdaBoost
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)
gb_reg = GradientBoostingRegressor(n_estimators=100, random_state=42)
ada_reg = AdaBoostRegressor(n_estimators=100, random_state=42)

# Fitting the ensemble model to the training data
rf_reg.fit(X_train.reshape(-1, X_train.shape[1]), y_train)
gb_reg.fit(X_train.reshape(-1, X_train.shape[1]), y_train)
ada_reg.fit(X_train.reshape(-1, X_train.shape[1]), y_train)

# Predicting the test data using ensemble approach
y_pred_rf = rf_reg.predict(X_test.reshape(-1, X_test.shape[1]))
y_pred_gb = gb_reg.predict(X_test.reshape(-1, X_test.shape[1]))
y_pred_ada = ada_reg.predict(X_test.reshape(-1, X_test.shape[1]))
y_pred_ensemble = (y_pred_rf + y_pred_gb + y_pred_ada) / 3

# Calculating the root mean squared error using ensemble approach
rmse_ensemble = np.sqrt(mean_squared_error(y_test, y_pred_ensemble))
print('Ensemble Root Mean Squared Error:', rmse_ensemble)


In [ ]:

import pickle
  
# Save the trained model as a pickle string.
saved_model = pickle.dumps(model)


from joblib import Parallel, delayed
import joblib
  
  
# Save the model as a pickle in a file
joblib.dump(ada_reg, 'model_ensemble_full.pkl')


In [ ]:
import tensorflow as tf

y_pred = model.predict(X_test)

# Calculating the root mean squared error
rmse = np.sqrt(mean_squared_error(y_test, y_pred_ada))
print('Root Mean Squared Error:', rmse)
# Calculating accuracy
accuracy = tf.keras.metrics.Accuracy()
accuracy.update_state(tf.math.round(y_test), tf.math.round(y_pred_ada))
acc = accuracy.result().numpy()
print('Accuracy:', acc)

In [ ]:

import pickle
  
# Save the trained model as a pickle string.
saved_model = pickle.dumps(model)


from joblib import Parallel, delayed
import joblib
  
  
# Save the model as a pickle in a file
joblib.dump(saved_model, 'model2_lstm_full.pkl')

In [ ]:
print(y_test)

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
import joblib

# Defining the list of tickers
tickers = ['SPY']

# Collecting the data
data = yf.download(tickers, start='2022-01-01', end='2023-04-12')

# Cleaning the data
data.dropna(inplace=True)

# Normalizing the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

# Splitting the data into training and testing sets
train_size = int(len(scaled_data) * 0.8)
test_size = len(scaled_data) - train_size
train_data, test_data = scaled_data[0:train_size,:], scaled_data[train_size:len(scaled_data),:]

# Defining the function to create the dataset with look-back
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

# Creating the dataset with look-back
look_back = 60
X_test, y_test = create_dataset(test_data, look_back)

# Reshaping the input to be [samples, time steps, features]
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Load the saved LSTM model
#lstm_model = joblib.load('filename.pkl')

# Predicting the test data using the LSTM model
#y_pred_lstm = lstm_model.predict(X_test)
#
# Load the saved ensemble model
#ensemble_model = joblib.load('model.pkl')
ensemble_model = joblib.load('model2_ensemble_full.pkl')


# Predicting the test data using the ensemble model
y_pred_ensemble = ensemble_model.predict(X_test.reshape(-1, X_test.shape[1]))

# Inverse scaling the predicted values to get the actual stock prices
#y_pred_lstm = scaler.inverse_transform(y_pred_lstm)
y_pred_ensemble = scaler.inverse_transform(y_pred_ensemble.reshape(-1, 1))

# Print the predicted next week's stock prices
#print('Predicted next week stock prices using LSTM model:\n', y_pred_lstm[-1])
print('Predicted next week stock prices using Ensemble model:\n', y_pred_ensemble[-1])
